# Introduction to ASE and GPAW

ASE is a module designed for working with atoms. It uses the units of Ångstrom (Å) for length and electron volts (eV) for energy.

In essence, ASE contains the `Atoms` object, which is a collection om `Atom` object - thus, when we loop through the `Atoms` object, we get an `Atom` object. The `Atoms` object can then be associated with a so-called `calculator` object, which is just an object which knows how to calculate energies and forces, e.g. GPAW.

<img src="images/ase-outline.png">

ASE and GPAW are quite complex modules, but there are good tutorials for doing many things, which can be found on their respective wiki pages.

https://wiki.fysik.dtu.dk/ase/

https://wiki.fysik.dtu.dk/gpaw/



## Contents
If you already are proficient in a topic you can skip it and move on to the next.

- [ASE](#ase)
- [GPAW](#gpaw)
- [Submitting calculations](#submit)


## ASE
<a id="ase"></a>


In [1]:
# CO molecule with a bond length of 1.1 Å
from ase import Atoms
d = 1.1
atoms = Atoms('CO', positions=[[0, 0, 0], [0, 0, d]])


ASE contains tools to visualize the system. This opens a new window for viewing the atoms

In [2]:
from ase.visualize import view
view(atoms)

We can loop through the `Atoms` object to get `Atom` objects

In [3]:
print(atoms)
print(atoms.positions)
for atom in atoms:
    print(atom)
    print(atom.index, atom.position)

Atoms(symbols='CO', pbc=False)
[[0.  0.  0. ]
 [0.  0.  1.1]]
Atom('C', [0.0, 0.0, 0.0], index=0)
0 [0. 0. 0.]
Atom('O', [0.0, 0.0, 1.1], index=1)
1 [0.  0.  1.1]


As you can see, the first print statement is `Atoms`, which contains more than a single atom, while the `Atom` object only contains 1 atom. Both types of objects have variables that can be accessed directly (positions, magmoms, ...)


Let's try to setup a periodic structure.

In [4]:
d = 2.9
L = 10
wire = Atoms('Au', positions=[[0, L / 2, L / 2]],
             cell=[d, L, L],  # unit cell lengths
             pbc=[1, 0, 0])  # periodic boundary conditions
# let's try and repeat it and visualize primitive and repeated
wire10 = wire * (10, 1, 1)
view([wire, wire10])

Let's setup a surface using one of the utility functions in [`ase.build`](https://wiki.fysik.dtu.dk/ase/dev/ase/build/build.html#module-ase.build), add an [adsorbate](https://wiki.fysik.dtu.dk/ase/dev/ase/build/surface.html#adding-adsorbates), [fix](https://wiki.fysik.dtu.dk/ase/dev/ase/constraints.html#the-fixatoms-class) the "bulk" atoms and finally do a geometrical [relaxation](https://wiki.fysik.dtu.dk/ase/dev/ase/optimize.html#module-ase.optimize)

In [5]:
# Create the slab
from ase.build import fcc100

slab = fcc100('Cu',
              size=(3, 3, 3),
              vacuum=7)
view(slab)

In [6]:
# Add an adsorbate
from ase.build import add_adsorbate

add_adsorbate(slab, adsorbate='Cu',
              height=3.0,
              position='ontop')

view(slab)

In [7]:
# Constrain the lower layers of the slab, they are the bulk
from ase.constraints import FixAtoms

con = FixAtoms(mask=[atom.tag > 1 for atom in slab])
slab.set_constraint(con)

view(slab)

In [8]:
# Attach a calculator and relax the atomic positions
from ase.calculators.emt import EMT
from ase.optimize import BFGS

# The calculator
calc = EMT()
slab.set_calculator(calc)

# The optimizer
traj_file = 'Cu-slab-relax.traj'
opt = BFGS(slab, trajectory=traj_file)
opt.run(fmax=0.05)  # unit of force is eV/Å


      Step     Time          Energy         fmax
BFGS:    0 14:07:48       10.478859        1.4775
BFGS:    1 14:07:48       10.430330        1.5194
BFGS:    2 14:07:48       10.338298        1.5920
BFGS:    3 14:07:48       10.245728        1.6524
BFGS:    4 14:07:48       10.153012        1.6958
BFGS:    5 14:07:48       10.061012        1.7163
BFGS:    6 14:07:48        9.970931        1.7064
BFGS:    7 14:07:48        9.884688        1.6573
BFGS:    8 14:07:48        9.804814        1.5586
BFGS:    9 14:07:48        9.734398        1.3995
BFGS:   10 14:07:48        9.676834        1.1707
BFGS:   11 14:07:48        9.635117        0.8691
BFGS:   12 14:07:48        9.610043        0.5096
BFGS:   13 14:07:48        9.595613        0.8567
BFGS:   14 14:07:48        9.577820        1.0506
BFGS:   15 14:07:49        9.552948        1.1590
BFGS:   16 14:07:49        9.524542        1.1981
BFGS:   17 14:07:49        9.494331        1.1814
BFGS:   18 14:07:49        9.463939        1.1161
B

True

In [9]:
# view the steps of the relaxation
from ase.io import read

slab_relax = read(traj_file, index=':')

view(slab_relax)

In [10]:
# add small pertubation away from the symmetric position of the adsorbate
slab[-1].position += [.1, .1, 0]
opt.run(fmax=0.05)
slab_relax = read(traj_file, index=':')

view(slab_relax)


BFGS:   25 14:15:45        9.357123        0.1118
BFGS:   26 14:15:46        9.352098        0.1414
BFGS:   27 14:15:46        9.345959        0.1694
BFGS:   28 14:15:46        9.338699        0.1967
BFGS:   29 14:15:46        9.330312        0.2234
BFGS:   30 14:15:46        9.320804        0.2495
BFGS:   31 14:15:46        9.310187        0.2751
BFGS:   32 14:15:46        9.298475        0.3003
BFGS:   33 14:15:46        9.285670        0.3258
BFGS:   34 14:15:46        9.271763        0.3517
BFGS:   35 14:15:46        9.256738        0.3779
BFGS:   36 14:15:46        9.240592        0.4042
BFGS:   37 14:15:46        9.223325        0.4304
BFGS:   38 14:15:46        9.204947        0.4564
BFGS:   39 14:15:46        9.185469        0.4822
BFGS:   40 14:15:46        9.164926        0.5066
BFGS:   41 14:15:46        9.143419        0.5281
BFGS:   42 14:15:47        9.121092        0.5467
BFGS:   43 14:15:47        9.098061        0.5633
BFGS:   44 14:15:47        9.074415        0.5781


In [11]:
# get the energy out directly
print(slab.get_potential_energy())

8.624371628630778


## GPAW
<a id="gpaw"></a>

[GPAW](https://wiki.fysik.dtu.dk/gpaw/index.html) is a density functional theory code written primarily in Python. It is based on the projector augmented wave (PAW) method. 3 different methods to describe the wave functions; plane wave (`mode=pw`), linear combination of atomic orbitals (`mode=lcao`) and on a real-space uniform grids with the finite-difference approximation (`mode=fd`).

In [ ]:
# How to import the GPAW calculator
from gpaw import GPAW

calc = GPAW(h=0.24,
            mode='lcao',
            basis='sz(dzp)',
            xc='PBE')

Let's calculate the DFT adsorption energy of CO on Cu(100) ontop site.

Adsorption energy is defined: $E_{ads} = E_{Cu+CO} - (E_{Cu} + E_{CO})$

In [ ]:
#%%writefile CO.py
# First check your structure is correct
from ase.build import molecule
from ase.optimize import BFGS
mol = molecule('CO')
mol.center(vacuum=10)

view(mol)


In [ ]:
# Then attach the calculator
from gpaw import GPAW
calc = GPAW(h=0.24,
            mode='lcao',
            basis='sz(dzp)',
            xc='PBE')

mol.set_calculator(calc)
opt = BFGS(mol, trajectory='CO.traj')
opt.run(fmax=0.05)  # unit of force is eV/Å



In [ ]:
#%%writefile Cu-slab.py
from ase.build import fcc100
from ase.build import add_adsorbate
from ase.build import molecule
from ase.constraints import FixAtoms
from gpaw import GPAW
from ase.optimize import BFGS

mol = molecule('CO')
slab = fcc100('Cu',
              size=(2, 2, 3),
              vacuum=7)
con = FixAtoms(mask=[atom.tag > 1 for atom in slab])
slab.set_constraint(con)

# calc = GPAW(h=0.24,
#             mode='lcao',
#             basis='sz(dzp)',
#             xc='PBE',
#             kpts=(6, 6, 1))

# calc.set(txt='slab.txt')
# slab.set_calculator(calc)
# opt = BFGS(slab, trajectory='Cu-clean.traj')
# opt.run(fmax=0.05)
add_adsorbate(slab, adsorbate=mol,
              height=1.8,
              position='ontop')

view(slab)

# calc.set(txt='slab-and-adsorbate.txt')
# opt = BFGS(slab, trajectory='Cu-adsorbate.traj')
# opt.run(fmax=0.05)



In [ ]:
e_Cu_ads = read('Cu-adsorbate.traj').get_potential_energy()
e_Cu = read('Cu-clean.traj').get_potential_energy()
e_mol = read('CO.traj').get_potential_energy()
print("DFT adsorption energy: E_ads = {:f.3}".format(e_Cu_ads - (e_Cu + e_mol)))

## Submitting calculations
<a id="submit"></a>

Often, DFT calculations take a while to run. A normal execution of a python program will only run on 1 core, however, that is very rarely ever enough to do any real simulations, so we turn to parallel execution of the programs.

We will submit our parallel calculations to a queueing system asking for a number of processors. Our computation will start when there is ressources available.

We have set up a program called `bsub.py`, which will take care of submitting to the queue. The syntax in the terminal is

```bash
bsub.py -t T -p NPROC myscript.py
```
which will submit `myscript.py` to the queue, requesting `NPROC` number of cores for the duration of `T` hours. So for example, it could look something like
```bash
bsub.py -t 1 -p 20 myscript.py
```
which would submit `myscript.py` to the queue for 1 hour on 20 processors. We can then look at our queue with the command 

```bash
bstat -u $USER
```
which gives us information about the jobs we currently have in the queue, whether they are waiting to start, running or completed. You can delete a job from the job with the command

```bash
qdel JOBID
```
where `JOBID` is the ID number of the job, which we can get with the `qstat` command above.

*Note* that terminal commands can be run in a Jupyter notebook cell by prefixing the command by an explamation mark (!). The submit command from above would be:

```
!bsub.py -t 1 -p 20 myscript.py
```